# ⚛️ H₂ Ground-State VQE — Noisy Ansatz Comparison

This notebook studies **VQE under realistic noise** via the API:

- `run_vqe_ansatz_comparison`
- `run_vqe`

The goal is to understand how noise affects:

- Ansatz stability
- Convergence behaviour
- The final **noisy density matrix**
- Basis-state populations

Noise model (identical for all ansatzes):
- **Depolarizing:** 5%
- **Amplitude damping:** 5%

These values are intentionally exaggerated to make behavioural differences clear.


In [31]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Always save VQE notebook plots to repo-root images/vqe/
VQE_IMG_DIR = Path('images') / 'vqe'
VQE_IMG_DIR.mkdir(parents=True, exist_ok=True)
import os, sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.core import run_vqe_ansatz_comparison, run_vqe

# Noise settings
depolarizing_prob = 0.05
amplitude_damping_prob = 0.05

seed = 0

# Fixed optimizer settings
optimizer_name = "Adam"
steps = 50
stepsize = 0.2


# 🔧 Part 1 — Noisy Ansatz Comparison

`run_vqe_ansatz_comparison(...)`

✔ Runs each ansatz under the same optimizer  
✔ Applies both noise channels  
✔ Handles reproducibility + caching  
✔ Saves a convergence plot  
✔ Returns final energies


In [ ]:
# Ansatzes to compare
ansatzes = [
    "UCC-S",
    "UCC-D",
    "UCCSD",
    "Minimal",
    "RY-CZ",
    "TwoQubit-RY-CNOT",
    "StronglyEntanglingLayers",
]

comparison = run_vqe_ansatz_comparison(
    molecule="H2",
    optimizer_name=optimizer_name,
    ansatzes=ansatzes,
    steps=steps,
    stepsize=stepsize,
    noisy=True,
    depolarizing_prob=depolarizing_prob,
    amplitude_damping_prob=amplitude_damping_prob,
    mapping="jordan_wigner",
    show=True,
    force=False,
)



# ⭐ Part 2 — Identify the Worst Ansatz

We define the *worst* ansatz as the one with the **highest final energy**.


In [ ]:
worst_ansatz = max(
    comparison["final_energies"],
    key=comparison["final_energies"].get
)

print(f"🏆 Worst ansatz under noise: {worst_ansatz}")


# 🔍 Part 3 — Full Noisy VQE Using the Worst Ansatz

We now run a complete noisy VQE with:

- Ansatz: worst performer
- Optimizer: Adam
- Depolarizing: 5%
- Amplitude damping: 5%

`run_vqe(...)` returns energies, metadata, and the full **noisy density matrix**
(because `noisy=True` uses `default.mixed`).


In [ ]:
res = run_vqe(
    molecule="H2",
    ansatz_name=worst_ansatz,
    optimizer_name=optimizer_name,
    steps=steps,
    stepsize=stepsize,
    noisy=True,
    depolarizing_prob=depolarizing_prob,
    amplitude_damping_prob=amplitude_damping_prob,
    seed=seed,
    plot=False,
    force=False,
)

rho = np.array(res["final_state_real"]) + 1j * np.array(res["final_state_imag"])
diag = np.diag(rho)

diag


# 🧮 Part 4 — Approximate Ket Representation

For mixed states, the diagonal entries of ρ correspond to **basis-state populations**.
We display only those above a small threshold.


In [ ]:
threshold = 1e-2
num_wires = int(np.log2(len(diag)))

terms = [
    f"({pop:.4f}|{idx:0{num_wires}b}⟩)"
    for idx, pop in enumerate(diag)
    if abs(pop) > threshold
]

ket_estimate = " + ".join(terms)

print("Approximate noisy state (from diagonal of ρ):")
print(f"|ψ⟩ ≈ {ket_estimate}")


# 📊 Part 5 — Basis-State Population Plot

The bar plot below visualizes **which computational states survive** after noise.


In [ ]:
indices = np.where(abs(diag) > threshold)[0]
vals = diag[indices]
labels = [f"|{i:0{num_wires}b}⟩" for i in indices]

plt.figure(figsize=(10, 5))
plt.bar(labels, vals.real, label="Real")
plt.bar(labels, vals.imag, bottom=vals.real, alpha=0.6, label="Imag")

plt.xlabel("Basis state")
plt.ylabel("Population")
plt.title(f"H₂ — Noisy VQE Ground State (Diag(ρ))\nWorst ansatz: {worst_ansatz}")
plt.legend()
plt.tight_layout()

out_name = f"H2_Noisy_Ground_State_{worst_ansatz.replace(' ', '_').replace('+','plus')}.png"
from vqe_qpe_common.plotting import save_plot
save_plot(str(out_name), kind="vqe", show=False)
plt.show()
